## Exercise 13.2 - Extended ML Fit  
Rewrite Exercise 13.1 so that the model is suitable for extended ML fitting.

Multiply:

the signal PDF by Nsig = 200 (range: 0-10000)  

the background PDF by Nbkg = 800 (range: 0-10000)

In [1]:
import ROOT

In [2]:
# declare observable 'x'
x = ROOT.RooRealVar("x", "x", 0, 10)

# build a Gaussian PDF called sig with  mean = 0 and sigma = 3
mean = ROOT.RooRealVar("mean", "mean of gaussian", 0)
sigma = ROOT.RooRealVar("sigma", "width of gaussian", 3)
sig = ROOT.RooGaussian("sig", "gaussian pdf", x, mean, sigma)

# add an exponential background component called bkg, defined as exp(-x/tau) with an initial value of tau = 10
# define the alpha/rate parameter of the exponential distribution
tau = ROOT.RooRealVar("tau", "mean life", 10, 0, 50)
rate = ROOT.RooFormulaVar("rate", "exp rate", "-1./tau", [tau])
bkg = ROOT.RooExponential("bkg", "background", x, rate)

[#0] WARNING:InputArguments -- The parameter 'sigma' with range [-inf, inf] of the RooGaussian 'sig' exceeds the safe range of (0, inf). Advise to limit its range.


In [3]:
# define nbkg and nsig as expected number of events for background and signal components
nsig = ROOT.RooRealVar("nsig", "Expected number of signal events", 200, 0, 10000)
nbkg = ROOT.RooRealVar("nbkg", "Expected number of background events", 800, 0, 10000)

# construct the composite model: 
#   model(x) = nsig * sig(x) + nbkg * bkg
model = ROOT.RooAddPdf("model", "composite model", ROOT.RooArgList(sig, bkg), ROOT.RooArgList(nsig, nbkg))

In [4]:
# visualize the pdf
c = ROOT.TCanvas()

xframe = x.frame()
xframe.SetTitle("Composite Model: Nsig*Signal + Nbkg*Background")

model.plotOn(xframe, ROOT.RooFit.Name("Composite Model"), ROOT.RooFit.LineColor(ROOT.kBlue))
model.plotOn(xframe, ROOT.RooFit.Components("sig"), ROOT.RooFit.Name("Signal"), ROOT.RooFit.LineColor(ROOT.kBlack), ROOT.RooFit.LineStyle(ROOT.kDashed))
model.plotOn(xframe, ROOT.RooFit.Components("bkg"), ROOT.RooFit.Name("Background"), ROOT.RooFit.LineColor(ROOT.kRed), ROOT.RooFit.LineStyle(ROOT.kDotted))

xframe.Draw()

legend = ROOT.TLegend(0.5, 0.7, 0.8, 0.8)
legend.AddEntry(xframe.findObject("Composite Model"), "Signal+Background", "l")
legend.AddEntry(xframe.findObject("Signal"), "Signal Component", "l")
legend.AddEntry(xframe.findObject("Background"), "Background Component", "l")
legend.Draw()

c.Draw()

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (sig)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: (rate)


In [5]:
# for extended ML use unbinned dataset
# generate data with the expected number of events from model = nsig + nbkg = 1000
data = model.generate(x)

# fit model to data, extended ML term automatically included
model.fitTo(data, ROOT.RooFit.Extended(True), ROOT.RooFit.PrintLevel(-1))

nsig.Print()
nbkg.Print()
mean.Print()
sigma.Print()
tau.Print()

c2 = ROOT.TCanvas("c2", "c2")

xframe2 = x.frame()
xframe2.SetTitle("Extended Maximum Likelihood Fit")

data.plotOn(xframe2, ROOT.RooFit.Name("Data"))
model.plotOn(xframe2, ROOT.RooFit.Name("ML Fit"))
model.plotOn(xframe2, ROOT.RooFit.Components("sig"), ROOT.RooFit.Name("Signal"), ROOT.RooFit.LineColor(ROOT.kBlack), ROOT.RooFit.LineStyle(ROOT.kDashed))
model.plotOn(xframe2, ROOT.RooFit.Components("bkg"), ROOT.RooFit.Name("Background"), ROOT.RooFit.LineColor(ROOT.kRed), ROOT.RooFit.LineStyle(ROOT.kDotted))

xframe2.Draw()

legend2 = ROOT.TLegend(0.5, 0.7, 0.8, 0.8)
legend2.AddEntry(xframe2.findObject("Data"), "Data", "lep")
legend2.AddEntry(xframe2.findObject("ML Fit"), "ML Fit", "l")
legend2.AddEntry(xframe2.findObject("Signal"), "Signal Component", "l")
legend2.AddEntry(xframe2.findObject("Background"), "Background Component", "l")

legend2.Draw()
c2.Draw()

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_modelData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization
RooRealVar::nsig = 452.976 +/- 41.8101  L(0 - 10000) 
RooRealVar::nbkg = 546.504 +/- 42.909  L(0 - 10000) 
RooRealVar::mean = 0 C  L(-INF - +INF) 
RooRealVar::sigma = 3 C  L(-INF - +INF) 
RooRealVar::tau = 49.3762 +/- 30.2213  L(0 - 50) 
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (sig)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF c